<a href="https://colab.research.google.com/github/ece-mdarifanzum/30-Days-of-Code-Hackerank/blob/main/Google_stock_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout

In [ ]:
data = pd.read_csv('/content/Google_train_data.csv')
data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1258 non-null   object 
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   object 
 5   Volume  1258 non-null   object 
dtypes: float64(3), object(3)
memory usage: 59.1+ KB


In [ ]:
data["Close"]=pd.to_numeric(data.Close,errors='coerce')
data = data.dropna()
trainData = data.iloc[:,4:5].values

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1149 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1149 non-null   object 
 1   Open    1149 non-null   float64
 2   High    1149 non-null   float64
 3   Low     1149 non-null   float64
 4   Close   1149 non-null   float64
 5   Volume  1149 non-null   object 
dtypes: float64(4), object(2)
memory usage: 62.8+ KB


In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
trainData = sc.fit_transform(trainData)
trainData.shape

(1149, 1)

In [ ]:
X_train = []
y_train = []

for i in range (60,1149): #60 : timestep // 1149 : length of the data
    X_train.append(trainData[i-60:i,0])
    y_train.append(trainData[i,0])

X_train,y_train = np.array(X_train),np.array(y_train)

In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1)) #adding the batch_size axis
X_train.shape

(1089, 60, 1)

In [ ]:
model = Sequential()

model.add(LSTM(units=100, return_sequences = True, input_shape =(X_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=100, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=100, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units=100, return_sequences = False))
model.add(Dropout(0.2))

model.add(Dense(units =1))
model.compile(optimizer='adam',loss="mean_squared_error")

In [ ]:
hist = model.fit(X_train, y_train, epochs = 20, batch_size = 32, verbose=2)

Epoch 1/20
35/35 - 17s - loss: 0.0510 - 17s/epoch - 477ms/step
Epoch 2/20
35/35 - 6s - loss: 0.0120 - 6s/epoch - 183ms/step
Epoch 3/20
35/35 - 8s - loss: 0.0116 - 8s/epoch - 228ms/step
Epoch 4/20
35/35 - 6s - loss: 0.0127 - 6s/epoch - 185ms/step
Epoch 5/20
35/35 - 8s - loss: 0.0083 - 8s/epoch - 225ms/step
Epoch 6/20
35/35 - 6s - loss: 0.0087 - 6s/epoch - 183ms/step
Epoch 7/20
35/35 - 8s - loss: 0.0067 - 8s/epoch - 221ms/step
Epoch 8/20
35/35 - 6s - loss: 0.0102 - 6s/epoch - 183ms/step
Epoch 9/20
35/35 - 8s - loss: 0.0060 - 8s/epoch - 220ms/step
Epoch 10/20
35/35 - 6s - loss: 0.0059 - 6s/epoch - 179ms/step
Epoch 11/20
35/35 - 8s - loss: 0.0056 - 8s/epoch - 239ms/step
Epoch 12/20
35/35 - 6s - loss: 0.0064 - 6s/epoch - 179ms/step
Epoch 13/20
35/35 - 8s - loss: 0.0057 - 8s/epoch - 220ms/step
Epoch 14/20
35/35 - 6s - loss: 0.0059 - 6s/epoch - 184ms/step
Epoch 15/20
35/35 - 8s - loss: 0.0066 - 8s/epoch - 225ms/step
Epoch 16/20
35/35 - 6s - loss: 0.0059 - 6s/epoch - 181ms/step
Epoch 17/20
35/

In [ ]:
plt.plot(hist.history['loss'])
plt.title('Training model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
testData = pd.read_csv('Google_test_data.csv')
testData["Close"]=pd.to_numeric(testData.Close,errors='coerce')
testData = testData.dropna()
testData = testData.iloc[:,4:5]
y_test = testData.iloc[60:,0:].values
#input array for the model
inputClosing = testData.iloc[:,0:].values
inputClosing_scaled = sc.transform(inputClosing)
inputClosing_scaled.shape
X_test = []
length = len(testData)
timestep = 60
for i in range(timestep,length):
    X_test.append(inputClosing_scaled[i-timestep:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
X_test.shape

(192, 60, 1)

In [ ]:
y_pred = model.predict(X_test)
y_pred

6/6 [==============================] - 2s 74ms/step


array([[1.2064555],
       [1.2078861],
       [1.216804 ],
       [1.231618 ],
       [1.2457236],
       [1.2504436],
       [1.2441345],
       [1.231419 ],
       [1.2208493],
       [1.215489 ],
       [1.207881 ],
       [1.1968907],
       [1.1862442],
       [1.1760641],
       [1.1713274],
       [1.1724184],
       [1.184992 ],
       [1.2078596],
       [1.2367824],
       [1.2680643],
       [1.2870716],
       [1.2927068],
       [1.2824254],
       [1.2609696],
       [1.2372531],
       [1.2193002],
       [1.2095369],
       [1.2036208],
       [1.1944948],
       [1.1831957],
       [1.1711437],
       [1.157916 ],
       [1.1396216],
       [1.116997 ],
       [1.1043398],
       [1.1072254],
       [1.1222688],
       [1.1431407],
       [1.1658612],
       [1.1817437],
       [1.1972399],
       [1.2141134],
       [1.232473 ],
       [1.2499039],
       [1.2631998],
       [1.2685657],
       [1.2638178],
       [1.2569783],
       [1.2531188],
       [1.2549658],


In [ ]:
predicted_price = sc.inverse_transform(y_pred)

In [ ]:
plt.plot(y_test, color = 'red', label = 'Actual Stock Price')
plt.plot(predicted_price, color = 'green', label = 'Predicted Stock Price')
plt.title('Google stock price prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()